In [1]:
import nltk                             
import matplotlib.pyplot as plt         
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer  
import random as rd
import string
import re
import math

In [2]:
data_url = 'cbchealth.txt'
listed_tweets = open(data_url, "r", encoding="utf8")
tweets = list(listed_tweets)

for i in range(len(tweets)):
    tweets[i] =re.sub(r'^RT[\s]+', '', tweets[i])  #bysheel elretweets
    tweets[i] = re.sub(r'https?:\/\/.*[\r\n]*', '', tweets[i])  #bysheel el links
    tweets[i] = re.sub(r'#', '', tweets[i])   #bysheel elhashtags
    tweets[i] = re.sub(r'[0-9]', '', tweets[i])   #bysheel eldigits
    tweets[i] = tweets[i].lower()  #bthawel kolo lowercase
    tweets[i] = " ".join(filter(lambda x: x[0] != '@', tweets[i].split())) #btsheel ay kelma btebtedy bi @
    tweets[i] = re.sub(r' 0 ', 'zero',tweets[i]) 
    tweets[i]= re.sub(r'[^A-Za-z ]', '', tweets[i]) #remove all chars that are not letters or blank spaces
    tweets[i] = tweets[i].strip('\n') #btsheel elspace elzyada mn elakher
    tweets[i]=re.sub(r'([A-Za-z])\1{2,}', r'\1', tweets[i])  #bnsheel elhroof elmetkrara zy heyyyyy
    tweets[i] = " ".join(tweets[i].split()) #btsheel ay extra space
    tweets[i] = tweets[i][8:] #sheilna eldates

In [3]:
print(tweets)

['drugs need careful monitoring for expiry dates pharmacists say', 'sabra hummus recalled in us', 'us sperm bank sued by canadian couple didnt verify donor info executive says', 'manitoba pharmacists want clampdown on tylenol sales', 'mom of spooked by vaccinations reverses stand but then kids get sick', 'hamilton police send mental health pros to the front lines with cops', 'wind turbine noise linked to only health issue annoyance', 'wild west of ecigarettes sparks debate over regulation', 'dementia patients sold unproven brainwave optimization', 'passengers on second chinavancouver flight in weeks at risk of measles', 'check expiry dates health canada advises after alesse birth control pill recall', 'hashtagging eating disorders help or hindrance in recovery', 'obama says memory of daughters preschool asthma attacks spurs climate change debate', 'women into healing accused of failing drug addicts by angry families', 'expired alesse birth control exposes deficiency', 'despite paying t

In [4]:
def CalcDistance(T1, T2):

    intersection = set(T1).intersection(T2)

    union = set().union(T1, T2)
    
    j = 1 - (len(intersection) / len(union))
    
    return j
   

In [5]:
def InitCentroids(tweets_list, k):
    
    centroids = []
    
    for i in range(k):
        random_tweet_idx = rd.randint(0, len(tweets_list) - 1)
        centroids.append(tweets_list[random_tweet_idx])
    
    return centroids

In [6]:
C = InitCentroids(tweets,3)
print(C)

['alberta confirms case of mad cow disease st in canada since', 'sask artist not letting autism hold her back', 'fentanylrelated deaths spark warning in prince george']


In [7]:
def AssignCluster(tweets_list, centroids):
    
    clusters = dict()

    for i in range(len(tweets_list)):
        selected_cluster = -1
        least_distance = 100000000
        for j in range(len(centroids)):
            if CalcDistance(centroids[j],tweets_list[i]) < least_distance:  #lw asghar tb2a heya tb3 elcent
                least_distance = CalcDistance(centroids[j],tweets_list[i])
                selected_cluster = j
            if centroids[j] == tweets_list[i]:   #lw same yb2a wa2feen 3nd elcent
                 selected_cluster = j
                 least_distance = 0
                 break
        if least_distance == 1:                    #lw wala haga same nedy elcent ay tweet random
            selected_cluster = rd.randint(0, len(centroids) - 1)
   
        clusters.setdefault( selected_cluster, []).append([tweets_list[i]]) #bn-assign elpoints blclusters
        last_tweet_idx = len(clusters.setdefault(selected_cluster , [])) - 1
        clusters.setdefault(selected_cluster , [])[last_tweet_idx].append(least_distance)
            
    return clusters

In [8]:
def UpdateCentroids(clusters):

    centroids = []

    for i in range(len(clusters)):   #hnshouf any pt a2rab li kol elpoints ely flcluster to be 3lcentroid elgdida
       
        MinDistSum = 10000000000
        SelectedCentroid = -1

    
        min_dis = [] 
        
        for t_1 in range(len(clusters[i])):
            
            min_dis.append([])
            DisSum = 0
          
            for t_2 in range(len(clusters[i])):
                
                if t_1 != t_2:
                    
                    if t_2 < t_1:
                        Distance = min_dis[t_2][t_1]     
                    else:
                        Distance = CalcDistance(clusters[i][t_1][0], clusters[i][t_2][0])
                        
                    min_dis[t_1].append(Distance)
                    DisSum += Distance
                    
                else:            #lw wa2feen 3nd nafs eltweet tbee3y tb2a eldis 0
                    
                    min_dis[t_1].append(0)

        
            if DisSum < MinDistSum:
                MinDistSum = DisSum
                SelectedCentroid = t_1

   
        centroids.append(clusters[i][SelectedCentroid][0])

    return centroids


In [9]:
def ComputeSSE(clusters):

    SSE = 0
    
    for i in range(len(clusters)):
        for j in range(len(clusters[i])):
            SSE = SSE + (clusters[i][j][1] ** 2)

    return SSE

In [10]:
def Converge(LastCentroid, NewCentroids):
    
    isConverged= True
    

    if len(LastCentroid) != len(NewCentroids):
        isConverged=False
        return False
    for i in range(len(NewCentroids)):
        if " ".join(NewCentroids[i]) != " ".join(LastCentroid[i]):
            isConverged=False
            return False

    return True

In [11]:
def run_kmeans(tweets_list, k, MaxIterations=50):
    
    IterationCount = 0
    LastCentroids = []
    centroids = InitCentroids(tweets_list, k)

   
    while (Converge(LastCentroids, centroids)) == False and (IterationCount < MaxIterations):

        print("** Iteration number " + str(IterationCount) + " **")

      
        clusters = AssignCluster(tweets_list, centroids)   #1  
        LastCentroids = centroids                          #2
        centroids = UpdateCentroids(clusters)              #3
        
        IterationCount = IterationCount + 1
    
    print("** K means Status: ")
    if (IterationCount == MaxIterations):
        print(" K means didn't converge! ")
    else:
        print(" K means converged! ")

    SSE = ComputeSSE(clusters)

    return clusters, SSE


In [12]:
if __name__ == '__main__':
   
    exp = 5
    k = 3
    
    for i in range(exp):

        print("Exp number: " + str((i + 1)) + " for k = " + str(k)) 

        clusters, sse =run_kmeans(tweets, k)


        for j in range(len(clusters)):
            
       
            print(str(j+1) + ": Size of cluster = ", str(len(clusters[j])) + " tweets")
             
            # for k in range(len(clusters[j])):
            #     print("k" + str(k) + ", " + (" ".join(clusters[j][k][0])))

        print("** SSE of this experiment : " + str(sse))
        print('\n')
        k = k + 1
        

Exp number: 1 for k = 3
** Iteration number 0 **
** Iteration number 1 **
** K means Status: 
 K means converged! 
1: Size of cluster =  666 tweets
2: Size of cluster =  1353 tweets
3: Size of cluster =  1722 tweets
** SSE of this experiment : 147.6595735900959


Exp number: 2 for k = 4
** Iteration number 0 **
** Iteration number 1 **
** K means Status: 
 K means converged! 
1: Size of cluster =  534 tweets
2: Size of cluster =  1561 tweets
3: Size of cluster =  930 tweets
4: Size of cluster =  716 tweets
** SSE of this experiment : 137.02002761447636


Exp number: 3 for k = 5
** Iteration number 0 **
** Iteration number 1 **
** Iteration number 2 **
** K means Status: 
 K means converged! 
1: Size of cluster =  962 tweets
2: Size of cluster =  667 tweets
3: Size of cluster =  426 tweets
4: Size of cluster =  1016 tweets
5: Size of cluster =  670 tweets
** SSE of this experiment : 134.71528479373472


Exp number: 4 for k = 6
** Iteration number 0 **
** Iteration number 1 **
** K means